In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/test_data.zip'

Archive:  /content/drive/MyDrive/test_data.zip
   creating: test_data/
   creating: test_data/blasti/
  inflating: test_data/blasti/DSC_6396.jpg  
  inflating: test_data/blasti/DSC_6397.jpg  
  inflating: test_data/blasti/DSC_6398.jpg  
  inflating: test_data/blasti/DSC_6399.jpg  
  inflating: test_data/blasti/DSC_6400.jpg  
  inflating: test_data/blasti/DSC_6401.jpg  
  inflating: test_data/blasti/DSC_6402.jpg  
  inflating: test_data/blasti/DSC_6403.jpg  
  inflating: test_data/blasti/DSC_6405.jpg  
  inflating: test_data/blasti/DSC_6406.jpg  
  inflating: test_data/blasti/DSC_6407.jpg  
  inflating: test_data/blasti/DSC_6408.jpg  
  inflating: test_data/blasti/DSC_6409.jpg  
  inflating: test_data/blasti/DSC_6410.jpg  
  inflating: test_data/blasti/DSC_6411.jpg  
   creating: test_data/bonegl/
  inflating: test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/bonegl/DSC_4588.jpg  
  inflating: test_data/bonegl/DSC_4589.jpg  
  inflating: test_data/bonegl/DSC_4590.jpg  
  inflating:

In [3]:
!unzip '/content/drive/MyDrive/train_data.zip'

Archive:  /content/drive/MyDrive/train_data.zip
   creating: train_data/
   creating: train_data/blasti/
  inflating: train_data/blasti/DSC_6382.jpg  
  inflating: train_data/blasti/DSC_6383.jpg  
  inflating: train_data/blasti/DSC_6384-2.jpg  
  inflating: train_data/blasti/DSC_6384.jpg  
  inflating: train_data/blasti/DSC_6385.jpg  
  inflating: train_data/blasti/DSC_6386.jpg  
  inflating: train_data/blasti/DSC_6387.jpg  
  inflating: train_data/blasti/DSC_6388.jpg  
  inflating: train_data/blasti/DSC_6389.jpg  
  inflating: train_data/blasti/DSC_6390.jpg  
  inflating: train_data/blasti/DSC_6391.jpg  
  inflating: train_data/blasti/DSC_6392.jpg  
  inflating: train_data/blasti/DSC_6393.jpg  
  inflating: train_data/blasti/DSC_6394.jpg  
  inflating: train_data/blasti/DSC_6395.jpg  
   creating: train_data/bonegl/
  inflating: train_data/bonegl/DSC_4570.jpg  
  inflating: train_data/bonegl/DSC_4571.jpg  
  inflating: train_data/bonegl/DSC_4583.jpg  
  inflating: train_data/bonegl/DS

In [4]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_path = "/content/train_data"
test_path = "/content/test_data"

In [6]:
train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255) 

In [7]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [8]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [9]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Dropout

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [11]:
# Adding the preprocessing layer to the front of vgg

vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [12]:
for layer in vgg.layers:
  print(layer)

In [13]:
# Train model with existing weights

for layer in vgg.layers:
  layer.trainable=False

In [14]:
x = Flatten()(vgg.output)

In [15]:
# output layer
prediction = Dense(16,activation='softmax')(x)

In [16]:
# Create Vgg16 model

model = Model(inputs=vgg.input,outputs=prediction)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
len(train),len(test)

(7, 8)

In [20]:
model.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),
                    validation_steps=len(test))

<ipython-input-20-240d737b1fe6>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),


Epoch 1/10
7/7 [==============================] - 105s 14s/step - loss: 3.1300 - accuracy: 0.1867 - val_loss: 2.9165 - val_accuracy: 0.2675
Epoch 2/10
7/7 [==============================] - 88s 14s/step - loss: 1.3713 - accuracy: 0.6067 - val_loss: 2.5544 - val_accuracy: 0.2994
Epoch 3/10
7/7 [==============================] - 87s 14s/step - loss: 0.8488 - accuracy: 0.7533 - val_loss: 2.6097 - val_accuracy: 0.3885
Epoch 4/10
7/7 [==============================] - 86s 14s/step - loss: 0.4821 - accuracy: 0.8600 - val_loss: 2.9371 - val_accuracy: 0.2994
Epoch 5/10
7/7 [==============================] - 121s 19s/step - loss: 0.3144 - accuracy: 0.9000 - val_loss: 3.0235 - val_accuracy: 0.3885
Epoch 6/10
7/7 [==============================] - 89s 14s/step - loss: 0.2435 - accuracy: 0.9333 - val_loss: 3.0369 - val_accuracy: 0.3376
Epoch 7/10
7/7 [==============================] - 87s 13s/step - loss: 0.1802 - accuracy: 0.9467 - val_loss: 2.8329 - val_accuracy: 0.3885
Epoch 8/10
7/7 [=========

In [21]:
img1 = image.load_img('/content/bird_samp.jpg',target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti', 'bonegl', 'brhkyt', 'cbrtsh', 'cmnmyn', 'gretit', 'hilpig', 'himbul', 'himgri', 'hsparo', 'indvul', 'jglowl', 'lbicrw', 'mgprob', 'rebimg', 'wcrsrt']
print(output[pred])

1/1 [==============================] - 1s 862ms/step
4
cmnmyn
